# Máquina de Pila para GoxLang IR

## Objetivo General

Construir una máquina virtual basada en pila que interprete y ejecute programas representados en el IR de GoxLang, soportando operaciones aritméticas, control de flujo, acceso a memoria, funciones, y variables.

### 1. Requisitos Funcionales

#### 1.1. Ejecución de Instrucciones IR

La máquina debe ser capaz de interpretar y ejecutar instrucciones del lenguaje intermedio (IR) especificado, incluyendo:
* Aritmética entera (ADDI, SUBI, etc.)
* Aritmética en punto flotante (ADDF, SUBF, etc.)
* Lógica y comparación (ANDI, ORI, EQI, NEF, etc.)
* Conversión entre tipos (ITOF, FTOI)
* Acceso a memoria (PEEKI, POKEI, PEEKF, POKEF, PEEKB, POKEB)
* Carga y almacenamiento de variables (LOCAL_GET, GLOBAL_SET, etc.)
* Funciones y retorno (CALL, RET)
* Control de flujo estructurado (IF, ELSE, ENDIF, LOOP, CBREAK, ENDLOOP)
* Expansión de memoria (GROW)
* Entrada/salida (PRINTI, PRINTF, PRINTB)

#### 1.2. Memoria
* Memoria lineal (byte-addressable) inicializable y extensible.
* Soporte para operaciones de lectura/escritura enteras, flotantes y bytes.

#### 1.3. Pila de datos
* Una pila que almacene operandos y resultados intermedios.
* Soporte para tipos int, float, char (como enteros) y bool (como int).

#### 1.4. Variables y entorno
* Variables locales y globales, con acceso por nombre.
* Soporte para scopes (entornos) por función.

#### 1.5. Llamadas a funciones
* Stack de activación con:
  * Retorno
  * Variables locales
  * Punto de continuación


### 2. Arquitectura Propuesta

#### 2.1. Componentes principales
* StackMachine
* Ejecuta instrucciones IR paso a paso.
* Administra pila, memoria, variables, funciones.
* Memory
* Memoria lineal de bytes.
* Métodos para leer/escribir enteros y flotantes.
* CallFrame
* Representa un entorno de ejecución para una función.
* Instruction Parser
* Carga un programa IR desde texto o lista de instrucciones.

Este notebook implementa una máquina virtual de pila capaz de interpretar un subconjunto del lenguaje intermedio (IR) de GoxLang. En esta primera fase, soporta tipos enteros y operaciones básicas como `CONSTI`, `ADDI`, `PRINTI`, y `RET`.

In [ ]:
class StackMachine:
    def __init__(self):
        self.stack = []                       # Pila principal
        self.memory = [0] * 1024              # Memoria lineal
        self.globals = {}                     # Variables globales
        self.locals_stack = []                # Stack de variables locales por función
        self.call_stack = []                  # Stack de retorno
        self.functions = {}                   # Diccionario de funciones
        self.pc = 0                           # Contador de programa
        self.program = []                     # Programa IR cargado
        self.running = False

    def load_program(self, program):
        self.program = program

    def run(self):
        self.pc = 0
        self.running = True
        while self.running and self.pc < len(self.program):
            instr = self.program[self.pc]
            opname = instr[0]
            args = instr[1:] if len(instr) > 1 else []
            method = getattr(self, f"op_{opname}", None)
            if method:
                method(*args)
            else:
                raise RuntimeError(f"Instrucción desconocida: {opname}")
            self.pc += 1

    def op_CONSTI(self, value):
        self.stack.append(('int', value))

    def op_ADDI(self):
        b_type, b = self.stack.pop()
        a_type, a = self.stack.pop()
        if a_type == b_type == 'int':
            self.stack.append(('int', a + b))
        else:
            raise TypeError("ADDI requiere dos enteros")

    def op_PRINTI(self):
        val_type, value = self.stack.pop()
        if val_type == 'int':
            print(value)
        else:
            raise TypeError("PRINTI requiere un entero")

    def op_RET(self):
        self.running = False

### Programa de ejemplo (IR)

In [ ]:
program = [
    ('CONSTI', 10),
    ('CONSTI', 20),
    ('ADDI',),
    ('PRINTI',),
    ('RET',),
]

vm = StackMachine()
vm.load_program(program)
vm.run()

30


## Bibliografía recomendada
- **Virtual Machines: Versatile Platforms for Systems and Processes** – Jim Smith & Ravi Nair
- **Crafting Interpreters** – Robert Nystrom ([https://craftinginterpreters.com](https://craftinginterpreters.com))
- **Engineering a Compiler** – Keith D. Cooper & Linda Torczon
- **Writing a Simple Stack Machine in Python** – David Beasley
- **Structure and Interpretation of Computer Programs (SICP)** – Harold Abelson & Gerald Jay Sussman
- Documentación oficial de Python: [https://docs.python.org/3/library](https://docs.python.org/3/library)